# Feature Engineering Notebook

## Objectives

- Engineer features for Regression and Decision Tree models

## Inputs

- outputs/datasets/cleaned/TrainSetCleaned.csv
- outputs/datasets/cleaned/TestSetCleaned.csv

## Outputs

- generate a list with variables to engineer

---

## Change working directory
Change current working directory to its parent

In [1]:
import os 
cwd = os.getcwd()
cwd

'/workspaces/heritage-housing/jupyter_notebooks'

In [2]:
os.chdir(os.path.dirname(cwd))
print("You set a new current working directory")

You set a new current working directory


In [3]:
cwd = os.getcwd()
cwd

'/workspaces/heritage-housing'

---

## Load cleaned data

In [5]:
import pandas as pd
train_set_path = "outputs/datasets/cleaned/TrainSetCleaned.csv"
TrainSet = pd.read_csv(train_set_path)
TrainSet.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GrLivArea,...,TotalBsmtSF,HouseAge,RemodAge,GarageAge,TotalSF,AboveGradeSF,IsRemodeled,Has2ndFlr,HasPorch,HasDeck
0,1828,0.0,3.0,Av,48,Missing,1774,774,Unf,1828,...,1822,18,18,18.0,3650.0,1828.0,0,0,0,0
1,894,0.0,2.0,No,0,Unf,894,308,Missing,894,...,894,63,63,63.0,1788.0,894.0,0,0,0,0
2,964,0.0,2.0,No,713,ALQ,163,432,Unf,964,...,876,104,19,104.0,1840.0,964.0,1,0,0,0


In [6]:
test_set_path = "outputs/datasets/cleaned/TestSetCleaned.csv"
TestSet = pd.read_csv(test_set_path)
TestSet.head(3)

,1stFlrSF,2ndFlrSF,BedroomAbvGr,BsmtExposure,BsmtFinSF1,BsmtFinType1,BsmtUnfSF,GarageArea,GarageFinish,GrLivArea,...,TotalBsmtSF,HouseAge,RemodAge,GarageAge,TotalSF,AboveGradeSF,IsRemodeled,Has2ndFlr,HasPorch,HasDeck
0,2515,0.0,4.0,No,1219,Rec,816,484,Missing,2515,...,2035,68,50,50.0,4550.0,2515.0,1,0,0,0
1,958,620.0,3.0,No,403,BLQ,238,240,Unf,1578,...,806,84,75,84.0,2384.0,1578.0,1,1,0,0
2,979,224.0,3.0,No,185,LwQ,524,352,Unf,1203,...,709,75,75,75.0,1912.0,1203.0,0,1,0,0
